## Data Cleaning
### Context
The dataset is comprised of information about songs (tracks), music genres, albums, and artists as aggregated from the Free Music Archive (FMA). In particular, this data was used in an academic study on music information retrieval; both the paper and data is accessible from the following [github repository](https://github.com/mdeff/fma). The dataset itself contains 917 GB and 343 days of Creative Commons-licensed audio from 106,574 tracks from 16,341 artists and 14,854 albums, arranged in a hierarchical taxonomy of 161 genres. For the purposes of this project, we are only concerned with a subset of this data that actually has documentation of specific measurable musical characteristics such as tempo, energy, and danceability, to name a few. 

### Cleaning
We go through various steps to clean the data across the several files provided from the FMA. Much of the cleaning and data manipulation pertains to text editing and ensuring data is consistent across dataframes which were then loaded into a MySQL relation database. 

### Data Storage
MySQL database; see [Schema](#schema).

In [1]:
# load necessary libraries
import os
import sqlalchemy
import numpy as np
import pandas as pd

from dotenv import load_dotenv


# load variables from .env file into set of environment variables
load_dotenv()

# get environment variables for database connection
hostname = os.getenv("HOSTNAME")
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
database = os.getenv("DATABASE")
port = 3306

# create db connection str and MySQL engine pool
connect_str = f"mysql+pymysql://{username}:{password}@{hostname}:3306/{database}"
engine = sqlalchemy.create_engine(connect_str)

In [2]:
# load song/track characteristics data
echo_df = pd.read_csv("../Data/fma_metadata_updated/echonest_updated.csv")

# only keep desired columns
echo_df = echo_df.iloc[:, :9]

# show head
echo_df.head()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
0,2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661
1,3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240
2,5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661
3,10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590
4,134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072


In [3]:
# load data for songs/tracks
tracks_df = pd.read_csv("../Data/fma_metadata_updated/raw_tracks.csv")

# filter and only keep desired columns
tracks_df = tracks_df.filter([
    "track_id",
    "album_id",
    "artist_id",
    "track_duration",
    "track_explicit",
    "track_favorites",
    "track_listens",
    "track_title",
    "track_genres"
])

# inner join tracks with characteristics; only keep songs with characteristic information
tracks_echo_df = pd.merge(
    left=echo_df,
    right=tracks_df,
    how="inner",
    on="track_id"
)

# rename columns
tracks_echo_df.rename(
    {
        "track_duration": "duration",
        "track_explicit": "explicit",
        "track_favorites": "favorites",
        "track_listens": "listens",
        "track_title": "title",
        "track_genres": "genre_id"
    }, 
    axis=1, 
    inplace=True
)

# some preliminary data cleaning and data type conversions
tracks_echo_df["duration"] = tracks_echo_df.duration.apply(lambda x: int(x.split(":")[0])*60 + int(x.split(":")[1])) # convert song duration from min:sec format to seconds
tracks_echo_df["album_id"] = tracks_echo_df.album_id.apply(int) # convert to int datatype
tracks_echo_df["explicit"] = tracks_echo_df["explicit"].replace({"Radio-Unsafe": 1, "Radio-Safe": 0, np.nan: -1}) # replace with levels 1 being explicit, 0 be not explicit, and -1 being unsure
tracks_echo_df["genre_id"] = tracks_echo_df.genre_id.apply(lambda x: [genre["genre_id"] for genre in eval(x)]) # extract genre_id from list of genre dicts

# show head
tracks_echo_df.head()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,album_id,artist_id,duration,explicit,favorites,listens,title,genre_id
0,2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,1,1,168,1.0,2,1293,Food,[21]
1,3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,1,1,237,1.0,1,514,Electric Ave,[21]
2,5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,1,1,206,1.0,6,1151,This World,[21]
3,10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,6,6,161,0.0,178,50135,Freeway,[10]
4,134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,1,1,207,1.0,3,943,Street Music,[21]


In [4]:
# load data for genres
genres_df = pd.read_csv("../Data/fma_metadata_updated/genres_updated.csv")

# map tracks with multiple genres to individual rows
tracks_genres_df = tracks_echo_df.explode("genre_id")
tracks_genres_df["genre_id"] = tracks_genres_df["genre_id"].astype(int)

# match songs with only top-level genres (16 unique genres), eliminating niche genres (160+ total)
tracks_genres_df = pd.merge(
    left=pd.merge(                  # join genres with their top-level genre in the hiearchy
        left=genres_df,
        right=genres_df,
        how="inner",
        left_on="genre_id",
        right_on="top_level"
    ).iloc[:, :7],
    right=tracks_genres_df[["track_id", "genre_id"]],
    how="inner",
    left_on="genre_id_y",
    right_on="genre_id"
)

# drop unnecessary columns and rename others
tracks_genres_df = tracks_genres_df.drop(["genre_id_y","genre_id", "top_level_x", "parent_x"], axis=1)
tracks_genres_df = tracks_genres_df.drop_duplicates()
tracks_genres_df.rename(
    {
        "genre_id_x":"genre_id", 
        "#tracks_x":"num_tracks",
        "title_x": "title",
        "genre_color_x": "genre_color",
    },
    axis=1, 
    inplace=True
)

# create dataframe of genres
genres_df = tracks_genres_df.filter(["genre_id", "num_tracks", "title", "genre_color"]).drop_duplicates().reset_index(drop=True)

# create dataframe mapping tracks to top-level genres
tracks_genres_df = tracks_genres_df.filter(["track_id", "genre_id"]).drop_duplicates().reset_index(drop=True)

# create tracks dataframe, dropping a few erraneous observations
tracks_df = tracks_echo_df.drop("genre_id", axis=1)
tracks_df = tracks_df[~tracks_df.artist_id.isin([2442, 13401])]
tracks_df.loc[tracks_df.artist_id == 1680,"artist_id"] = 4542

In [5]:
genres_df.head()

,genre_id,num_tracks,title,genre_color
0,2,5271,International,#CC3300
1,3,1752,Blues,#000099
2,4,4126,Jazz,#990099
3,5,4106,Classical,#8A8A65
4,8,868,Old-Time / Historic,#665666


In [6]:
tracks_genres_df.head()

,track_id,genre_id
0,853,2
1,1082,2
2,1384,2
3,1680,2
4,1681,2


In [7]:
tracks_df.head()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,album_id,artist_id,duration,explicit,favorites,listens,title
0,2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,1,1,168,1.0,2,1293,Food
1,3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,1,1,237,1.0,1,514,Electric Ave
2,5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,1,1,206,1.0,6,1151,This World
3,10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,6,6,161,0.0,178,50135,Freeway
4,134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,1,1,207,1.0,3,943,Street Music


In [8]:
# load artist data
artists_df = pd.read_csv("../Data/fma_metadata_updated/raw_artists.csv")

# filter and only keep desired columns
artists_df = artists_df.filter(["artist_id","artist_favorites", "artist_name"])

# only keep artists that have made atleast one track
artists_df = artists_df[artists_df["artist_id"].isin(tracks_df.artist_id)]

# rename columns
artists_df.rename({"artist_favorites":"favorites", "artist_name":"name"}, axis=1, inplace=True)
artists_df.reset_index(drop=True, inplace=True)

# drop a few erraneous observations
artists_df = artists_df[~artists_df.artist_id.isin([2442, 13401])]
artists_df.loc[artists_df.artist_id == 1680, "artist_id"] = 4542

# show head
artists_df.head()

,artist_id,favorites,name
0,1,9,AWOL
1,10,111,Lucky Dragons
2,100,8,Chandeliers
3,10001,13,Nick Rivera
4,10005,8,Blank Realm


In [9]:
# load album data
albums_df = pd.read_csv("../Data/fma_metadata_updated/raw_albums.csv")

# only keep albums which have a matching artist name
albums_df = albums_df[albums_df["artist_name"].isin(artists_df.name)]

# swap matching artist names with artist ids to comply with sql schema
# swap self-titled albums (s/t) with the artist name
albums_df = pd.merge(
    left=albums_df,
    right=artists_df[["name", "artist_id"]],
    how="inner",
    left_on="artist_name",
    right_on="name"
)
albums_df["album_title"] = albums_df.apply(lambda row: row["artist_name"] + "(s/t)" if row["album_title"].lower() == "s/t" else row["album_title"], axis=1)

# filter and only keep desired columns
albums_df = albums_df.filter([
    "album_id", 
    "album_date_released", 
    "album_favorites", 
    "album_title", 
    "album_tracks", 
    "album_listens", 
    "artist_id"
])

# rename columns
albums_df.rename({ 
    "album_date_released": "release_date", 
    "album_favorites": "favorites", 
    "album_title": "title", 
    "album_tracks": "num_tracks", 
    "album_listens": "listens", 
    },
    axis=1,
    inplace=True
)

# convert release date to datetime format
albums_df["release_date"] = pd.to_datetime(albums_df.release_date, format="%m/%d/%Y")
albums_df.reset_index(drop=True, inplace=True)

# show head
albums_df.head()

,album_id,release_date,favorites,title,num_tracks,listens,artist_id
0,1,2009-01-05,4,AWOL - A Way Of Life,7,6073,1
1,13756,2012-04-01,0,Jackin4Beats,11,4995,1
2,100,2009-01-09,0,On Opaque Things,4,5613,80
3,101,2007-01-01,0,Wooden Lake Sexual Diner,3,2180,80
4,10308,2011-10-03,0,"Live at WFMU on Liz Berg's Show on October 3, ...",9,3163,80


In [10]:
# remap id in each dataframe to coincide with sql table schemas
tracks_df.rename({"track_id": "id"},axis=1, inplace=True)
genres_df.rename({"genre_id": "id"},axis=1, inplace=True)
artists_df.rename({"artist_id": "id"},axis=1, inplace=True)
albums_df.rename({"album_id": "id"},axis=1, inplace=True)

In [11]:
### final cleanup based of dataframes

# string cleanup of artist names
artists_df["name"] = artists_df.name.apply(lambda x: x.split(",")[0].replace("&amp", "&").replace("(F.A.G.G)", "").replace("(Various)", "").replace("&;", "&").replace("(avec logo panthère)", "").strip())

# string cleanup of album names
albums_df["title"] = albums_df.title.apply(lambda x: x.strip())

# only keep tracks that have an associate artist
tracks_df = tracks_df[tracks_df.artist_id.isin(artists_df.id)]

# if a track's album id isn't contained in the album data set it to NA
tracks_df.loc[~tracks_df.album_id.isin(albums_df.id), "album_id"] = np.nan

# fix this track title as it is titled NaN and should not be treated as a missing value
tracks_df.loc[tracks_df.title.isna(), "title"] = 'NaN'

# only keep tuples which have a track_id that's still present in the tracks dataframe
tracks_genres_df = tracks_genres_df[tracks_genres_df.track_id.isin(tracks_df.id)]

# drop all albums without a track in the dataset and correct all track counts
albums_df = pd.merge(
    left=tracks_df[["id", "album_id"]].groupby(by="album_id", as_index=False).count(),
    right=albums_df,
    left_on="album_id",
    right_on="id",
    how="inner",
    suffixes=("_counts", None)
).drop(["album_id", "num_tracks"], axis=1).rename({"id_counts": "num_tracks"},axis=1)

In [12]:
# map sql table to pandas dataframes
tables = dict(
    Artists=artists_df,
    Albums=albums_df, 
    Tracks=tracks_df, 
    Genres=genres_df, 
    Track_Genres=tracks_genres_df
)

# insert pandas tables into relevant MySQL tables
for table, df in tables.items():
    insertions = df.to_sql(
        table,
        engine,
        if_exists="append",
        index=False,
        chunksize=1000
    )
    print(f"Pandas dataframe inserted into MySQL {table} table successfully with {insertions} tuples.")

Pandas dataframe inserted into MySQL Artists table successfully with 2865 tuples.
Pandas dataframe inserted into MySQL Albums table successfully with 2197 tuples.
Pandas dataframe inserted into MySQL Tracks table successfully with 13074 tuples.
Pandas dataframe inserted into MySQL Genres table successfully with 16 tuples.
Pandas dataframe inserted into MySQL Track_Genres table successfully with 17715 tuples.


In [13]:
# check that data was inserted correctly
with engine.connect() as connection:
    for table in tables.keys():
        print(f"{table}\n{'-'*len(table)}")
        query = sqlalchemy.text(f"SELECT * FROM {table} ORDER BY {'id' if table!='Track_Genres' else 'track_id'} ASC LIMIT 10")
        response = connection.execute(query)
        for row in response:
            print(row)
        print()

# dispose of db connection pool and close all connections
engine.dispose()

Artists
-------
(1, 9, 'AWOL')
(4, 10, 'Nicky Cook')
(6, 74, 'Kurt Vile')
(10, 111, 'Lucky Dragons')
(54, 11, 'Alec K. Redfearn & the Eyesores')
(56, 7, 'Amoebic Ensemble')
(60, 0, 'Arc and Sender')
(62, 2, 'Argumentix')
(66, 11, 'Au')
(68, 0, 'Bad News Bats')

Albums
------
(1, 1, 4, 6073, 6, datetime.date(2009, 1, 5), 'AWOL - A Way Of Life')
(4, 4, 2, 2710, 2, datetime.date(2009, 1, 6), 'Niris')
(6, 6, 4, 47632, 1, datetime.date(2008, 2, 6), 'Constant Hitmaker')
(60, 54, 1, 1304, 2, datetime.date(2009, 1, 16), 'Every Man For Himself')
(61, 54, 1, 1300, 1, datetime.date(2007, 5, 22), 'The Blind Spot')
(62, 54, 1, 845, 1, datetime.date(2005, 1, 25), 'The Quiet Room')
(64, 56, 0, 2014, 2, datetime.date(2009, 1, 6), 'Amoebiasis')
(65, 56, 1, 1446, 2, datetime.date(1995, 1, 1), 'Limbic Rage')
(69, 60, 0, 628, 2, datetime.date(2005, 6, 7), 'Arc and Sender')
(70, 60, 0, 197, 1, datetime.date(2009, 1, 6), 'unreleased demo')

Tracks
------
(2, 1, 1, 0.416675, 0.675894, 0.634476, 0.0106281, 0.

### Schema
Table: `Artists`
| Field       | Type        | Null | Key |
|-------------|-------------|------|-----|
| id          | int         | NO   | PRI |
| favorites   | int         | YES  |     |
| name        | varchar(60) | YES  |     |


Table: `Albums`
| Field        | Type         | Null | Key |
|--------------|--------------|------|-----|
| id           | int          | NO   | PRI |
| artist_id    | int          | NO   | MUL |
| favorites    | int          | YES  |     |
| listens      | int          | YES  |     |
| num_tracks   | int          | YES  |     |
| release_date | date         | YES  |     |
| title        | varchar(100) | YES  |     |


Table: `Tracks`
| Field            | Type         | Null | Key |
|------------------|--------------|------|-----|
| id               | int          | NO   | MUL |
| artist_id        | int          | NO   | MUL |
| album_id         | int          | YES  | MUL |
| acousticness     | float        | YES  |     |
| danceability     | float        | YES  |     |
| energy           | float        | YES  |     |
| instrumentalness | float        | YES  |     |
| liveness         | float        | YES  |     |
| speechiness      | float        | YES  |     |
| tempo            | float        | YES  |     |
| valence          | float        | YES  |     |
| duration         | int          | YES  |     |
| explicit         | int          | YES  |     |
| favorites        | int          | YES  |     |
| listens          | int          | YES  |     |
| title            | varchar(150) | YES  |     |

Table: `Genres`
| Field       | Type        | Null | Key |
|-------------|-------------|------|-----|
| id          | int         | NO   | PRI |
| num_tracks  | int         | YES  |     |
| title       | varchar(55) | YES  |     |
| genre_color | varchar(7)  | YES  |     |

Table: `Track_Genres`
| Field    | Type | Null | Key |
|----------|------|------|-----|
| track_id | int  | NO   | MUL |
| genre_id | int  | NO   | MUL |